In [ ]:
"""Please keep in mind that this whole document is only
to give a general idea of what to implement. 
These are not functioning codes."""

In [1]:
import nltk

In [2]:
import spacy

In [3]:
from gensim.parsing.preprocessing import remove_stopwords

In [4]:
from spacy.matcher import Matcher
from spacy.util import filter_spans

In [5]:
from elasticsearch import Elasticsearch

In [6]:
nlp = spacy.load("en_core_web_lg")

In [7]:
# draft : implementation step
######################################
# translate regular sentence to slang#
######################################

input_str = "Thanks for taking the time to post. It appears the making of a nation is missing."

In [8]:
str_doc = nlp(input_str)

In [69]:
len(str_doc)

18

In [70]:
print(str(list(str_doc.noun_chunks)))

[Thanks, the time, It, the making, a nation]


In [25]:
def noun_phrases_extract(nlp_result):
    # NOTE: the results of spacy for eg
    # .noun_chunks is constituted of elements 
    # rather than simple strings 
    noun_lst = nlp_result.noun_chunks
    return list(noun_lst)

In [10]:
def verb_phrases_extract(nlp_result):
    # under a different scheme
    # acknowledgement: 
    # https://stackoverflow.com/questions/47856247/extract-verb-phrases-using-spacy
    pattern = [[{'POS': 'VERB', 'OP': '?'},{'POS': 'ADV', 'OP': '*'}, # additional wildcard - match any text in between
               {'POS': 'VERB', 'OP': '+'}]]
    print(type(pattern))
    matcher = Matcher(nlp.vocab)
    matcher.add("Verb phrase", pattern)
    matches = matcher(nlp_result)
    spans = [nlp_result[start:end] for _, start, end in matches]
    return filter_spans(spans)
    # preserve the parsed positions of words so that the -ing forms
    # won't get messed up

In [26]:
noun_phrases_extract(nlp("I am really in need of a break."))

[I, need, a break]

In [ ]:
""" kopfschmerzen: how to translate "I am really in need of a break" to 
" I rnab" for eg if rnab=really need a break is some kind of legal slang. 
IDK if fuzzy search can do that and I hope urban dic has abbreviations 2"""

In [37]:
# NOTE: not finished.
def phrase_extraction(nlp_result):
    # lemmatize first, extract phrases later
    lst_sentence = []
    for token in nlp_result:
        print(token.pos_)
        word = token.lemma_
        lst_sentence.append(word)
    sentence = " ".join(lst_sentence)
    nlp_result = nlp(sentence)
    print(nlp_result)
    noun_list = noun_phrases_extract(nlp_result)
    verb_list = verb_phrases_extract(nlp_result)
    lst_phrase_temp = noun_list+verb_list
    return lst_phrase_temp

In [38]:
phrase_extraction(str_doc)
# doing an adequate job but can be better

NOUN
ADP
VERB
DET
NOUN
PART
VERB
PUNCT
PRON
VERB
DET
NOUN
ADP
DET
NOUN
AUX
VERB
PUNCT
thank for take the time to post . it appear the making of a nation be miss .
<class 'list'>


[the time, it, the making, a nation, thank, take, post, appear, miss]

In [66]:
"I love".split(" ")

['I', 'love']

In [64]:
verb_phrases_extract(str_doc)[0]
# NOTE: theres room for improvement in accuracy

<class 'list'>


'taking'

In [ ]:
"""To find and return subject of a preprocessed sentence."""
# I wrote the function based on which one may
# omit the subject of a sentence with a possibility. 
# This is not used now, as I still don't
# know how to calculate the weight of such omission.
def subject_find(doc):
    for token in doc:
        if "subj" in token.dep_:
            subtree = list(token.subtree)
            start = subtree[0].i
            end = subtree[-1].i + 1
            return doc[start:end]

In [ ]:
# do slang dictionary query with part of the input sentence
"""returns all possible k-word-shingle for the input sentence.
form: list, [[1-shingle][2-shingle]...[k-shingle]]""" 
def multiple_shingle(doc_str):
    doc = doc_str.split(" ")
    shingle_lst=[]
    for k in range(0,len(doc)):
        shingle_lst_temp = []
        for i in range(len(doc)-k):
            shingle_lst_temp.append(doc[i:k+i+1])
        shingle_lst.append(shingle_lst_temp)
    return shingle_lst

In [ ]:
multiple_shingle(input_str)

[[['I'], ['have'], ['something'], ['to'], ['say']],
 [['I', 'have'], ['have', 'something'], ['something', 'to'], ['to', 'say']],
 [['I', 'have', 'something'],
  ['have', 'something', 'to'],
  ['something', 'to', 'say']],
 [['I', 'have', 'something', 'to'], ['have', 'something', 'to', 'say']],
 [['I', 'have', 'something', 'to', 'say']]]

In [ ]:
# TODO: perform lemmatization to all the shingles with POS tag

In [ ]:
# TODO: ES search function on searching abbreviation of phrases
""" 
"""
def search_es_abbreviation(list_words):
    return abbreviation->str

In [ ]:
"""search in abbreviation database based on shingles,
return slang and the corresponding shingle."""

def abbreviation_search(shingle_lst):
    lst_search_result = []
    for i in range(len(shingle_lst)):
        list_temp = shingle_lst[i]
        for j in range(len(list_temp)):
            word_temp = list_temp[j]
            search_result = search_es_abbreviation(word_temp)
            # TODO: make it a fuzzy search
            if search_result: # when search result non empty
                # give list of paired (word, search result)
                lst_search_result.append((word_temp, search_result))
    return lst_search_result

In [ ]:
""" use Urban dictionary to perform fuzzy search.
TODO
NOTE: another possible way is to bring the word to the phrase,
namely extract keyword from urban dictionary&match it with the phrase."""
def UD_slang_search(phrase_list):
    return slang_list
    # formulation of slang_list: [(slang, word/phrase)]

In [ ]:
"""search in UD database based on phrases,
return slang and the corresponding phrase."""

def slang_search(phrase_lst):
    lst_search_result = []
    for i in range(len(phrase_lst)):
        list_temp = phrase_lst[i]
        for j in range(len(list_temp)):
            word_temp = list_temp[j]
            search_result = UD_slang_search(word_temp)
            # TODO: make it a fuzzy search
            if search_result: # when search result non empty
                # give list of paired (word, search result)
                lst_search_result.append((word_temp, search_result))
    return lst_search_result

In [ ]:
# TODO: ES search in emoji database and return 
# list of corresponding emojis
def emoji_search_es(word):
    return emoji_list->list
    # return a list of emoji searched
    # can be UTF-8 encoded or numbered according to order in dataset

In [34]:
"""remove stop words,
perform a word association based on twitter corpus.
perform ES query based on associated words.
return: corresponding word and associated emoji as list of tuple."""


def emoji_search(input_str):
    emoji_search_results = []
    en_stops = nltk.corpus.stopwords.words("english")
    word_list = [x.strip() for x in input_str.split(" ")]
    model = Word2Vec.load('model.bin')
    for word in word_list:
        emoji_search_temp = []
        if word not in en_stops:
            # perform association
            # perform emoji search
            list_of_emoji_original = emoji_search_es([word])
            emoji_search_temp = emoji_search_temp + list_of_emoji_original
            if not len(list_of_emoji_original):
                # if the origial word cannot be found
                association_list = model.wv.most_similar(word)
                for word_associated in association_list:
                # store in list_of_emoji
                    list_of_emoji = emoji_search_es(word_associated)
                    emoji_search_temp = emoji_search_temp + list_of_emoji
        emoji_search_results.append((word, emoji_search_temp))
    return emoji_search_results
    # TODO: can also based on api to reversedictionary.com
    # which gives associated words to noun phrases: 
    # 1. extract noun phrases with nlp().noun_chunks,2.search
    # the phrase up in reversedictionary.com, and
    # 3.perform query with ES in emoji dataset with obtained 
    # set of words.

In [ ]:
def emoji_search_alternative(word_lst):
    emoji_search_results = []
    for word in word_lst:
        emoji_search_temp = []
            # perform association
        association_list = model.wv.most_similar(word)
            # perform emoji search
        for word_associated in association_list:
            # store in list_of_emoji
            list_of_emoji = emoji_search_es(word_associated)
        # TODO: return word_associated with associated emoji
            emoji_search_temp = emoji_search_temp + list_of_emoji
        emoji_search_results.append((word, emoji_search_temp))
    return emoji_search_results

In [ ]:
"""The general function that would 
translate input to sentence with slang&emoji.
return&print translated sentence.
To be continued.
"""
def translation_style(input_str):
    # for each of the words perform emoji search first
    # if we want the precise results of emoji association,
    # it would be best to not perform association based on trivial words
    # -> STOP WORD REMOVAL
    emoji_list = emoji_search(input_str)
    # completely word based so we might just 
    # append the emoji after words.
    # another way is to only perform association 
    # when you encounter a certain grammartical structure
    # for eg noun phrase&verb phrase. 
    str_doc = nlp(input_str)
    word_list = []
    for word in str_doc:
        if word.pos_ == "VERB" or word.pos_ == "NOUN" or word.pos_ == "ADJ":
            word_list.append(word)
    emoji_list_alternative = emoji_search_alternative(word_list)
    # need to note the order of the words since same word
    # can appear multiple times as multiple POS
    phrase_list = phrase_extraction(str_doc)
    
    shingle_lst = multiple_shingle(input_str)
    # the problem here is that after translating input
    # to slangs the word formulation would differ.
    # ->should make sure which part translates to which
    abbrev_list = abbreviation_search(shingle_lst)
    # search for further slangs with the original sentence
    # and if the slang results overlap with the abbreviations
    # for eg got 2 slangs for same word
    # then use any one with equal probability
    slang_list = slang_search(phrase_list)
    # TODO: get the results together:
    # append the emojis->
    # see if the abbrev list clashes with the UD one->
    # decide the final list->
    # substitute words with slangs
    return slang_sentence


    
    
    